
<div style="
    background-color: #f7f7f7;
    background-image: url(''), url('') ;
    background-position: left bottom, right top;
    background-repeat: no-repeat,  no-repeat;
    background-size: auto 60px, auto 160px;
    border-radius: 5px;
    box-shadow: 0px 3px 1px -2px rgba(0, 0, 0, 0.2), 0px 2px 2px 0px rgba(0, 0, 0, 0.14), 0px 1px 5px 0px rgba(0,0,0,.12);">

<h1 style="
    color: #2a4cdf;
    font-style: normal;
    font-size: 2.25rem;
    line-height: 1.4em;
    font-weight: 600;
    padding: 30px 200px 0px 30px;"> 
        Compositional Based Property Prediction of Perovskites with CrabNet</h1>

<p style="
    line-height: 1.4em;
    padding: 30px 200px 0px 30px;">
        This notebook demonstrates how to use <a href="https://github.com/sparks-baird/CrabNet" target="_blank">CrabNet</a> to predict the bandgap of perovskites
        using data from the <a href="https://www.nature.com/articles/s41560-021-00941-3" target="_blank"><i>Perovskite Database</i></a> in NOMAD for training. The method can be
        extended to predict other properties of perovskites as well as using any of the 
        other methodologies avialble in the community for this task. To explore more of these methodologies 
        we recomment having a lot at <a href="https://matbench.materialsproject.org/" target="_blank">MatBench</a>, a benchmarking suite for these tasks in the 
        materials informatics community.
</p>

<p style="font-size: 1.25em; font-style: italic; padding: 5px 200px 30px 30px;">
    José A. Márquez, Siamak Nakhaie and Hampus Näsström
</p>
</div>

## Outline

- [Installations](#installations)
- [Retrieve Data using NOMAD API](#retrieve-data-using-nomad-api)
- [Save Data](#save-data)
- [Load data](#load-data)
- [EDA & Data Cleaning](#eda--data-cleaning)
  - [Insights from NOMAD GUI](#insights-from-nomad-gui)
  - [Remove NaNs](#remove-nans)
  - [Group Repeated Formulas](#group-repeated-formulas)
  - [Check Element Prevalence](#check-element-prevalence)
- [Build and Fit the Model](#build-and-fit-the-model)
- [Model evaluation](#model-evaluation)
- [Predict Bandgap from Individual Formulas](#predict-bandgap-from-individual-formulas)

## Installations

We will start by running a couple of pip installers. Skip this part if you have the libraries installed in your environment.

In [ ]:
! pip install torch
! pip install crabnet
! pip install pandas

## Retrieve Data using NOMAD API

We will now fetch Perovskite solar cell data from the NOMAD API.  If you already have the data, the **perovskite_bandgap_devices.csv** in the **data** folder, you can skip this part, and continue with Loading Data. Note that calling the database through the API may take a while.

In [ ]:
import requests

base_url = 'https://nomad-lab.eu/prod/v1/api/v1'

def extract_values(entry):
    try:
        bandgaps.append(entry['results']['properties']['electronic']['band_structure_electronic'][0]['band_gap'][0]['value'])
    except (KeyError, IndexError, TypeError):
        bandgaps.append('None')
    try:
        reduced_formulas.append(entry['results']['material']['chemical_formula_reduced'])
        iupac_formulas.append(entry['results']['material']['chemical_formula_iupac'])
        descriptive_formulas.append(entry['results']['material']['chemical_formula_descriptive'])
    except(KeyError, IndexError, TypeError):
        reduced_formulas.append('None')
        iupac_formulas.append('None')
        descriptive_formulas.append('None')
    try:
        vocs.append(entry['results']['properties']['optoelectronic']['solar_cell']['open_circuit_voltage'])
    except(KeyError, IndexError, TypeError):
        vocs.append('None')
    try:
        jscs.append(entry['results']['properties']['optoelectronic']['solar_cell']['short_circuit_current_density'])
    except(KeyError, IndexError, TypeError):
        jscs.append('None')
    try:
        ffs.append(entry['results']['properties']['optoelectronic']['solar_cell']['fill_factor'])
    except(KeyError, IndexError, TypeError):
        ffs.append('None')
    try:
        pces.append(entry['results']['properties']['optoelectronic']['solar_cell']['efficiency'])
    except(KeyError, IndexError, TypeError):
        pces.append('None')
    try:
        device_stack.append(entry['results']['properties']['optoelectronic']['solar_cell']['device_stack'])
    except(KeyError, IndexError, TypeError):
        device_stack.append('None')
    try:
        htl.append(entry['results']['properties']['optoelectronic']['solar_cell']['hole_transport_layer'])
    except(KeyError, IndexError, TypeError):
        htl.append('None')
    try:
        etl.append(entry['results']['properties']['optoelectronic']['solar_cell']['electron_transport_layer'])
    except(KeyError, IndexError, TypeError):
        etl.append('None')
    try:
        structural_dimensionality.append(entry['results']['material']['structural_type'])
    except(KeyError, IndexError, TypeError):
        structural_dimensionality.append('None')
    return bandgaps, reduced_formulas, iupac_formulas, descriptive_formulas, vocs, jscs, ffs, pces, device_stack, htl, etl, structural_dimensionality

bandgaps = []
vocs = []
jscs = []
ffs = []
pces = []
reduced_formulas = []
iupac_formulas = []
descriptive_formulas = []
htl = []
etl = []
device_stack = []
structural_dimensionality = []
page_after_value = None

while True:
    data = requests.post(f'{base_url}/entries/query', json={
        "owner": "visible",
        "aggregations": {},
        "query": {
            "and": [
                {"sections:all": ["nomad.datamodel.results.SolarCell"]}
            ]
        },
        'required': {
            'results': {
                'material': {
                    'chemical_formula_reduced': '*',
                    'structural_type': '*'},
                'properties': {
                    'optoelectronic': {
                        'band_gap': '*',
                        'solar_cell': {
                            'open_circuit_voltage': '*',
                            'short_circuit_current_density': '*',
                            'fill_factor': '*',
                            'efficiency': '*'}}},
            },
        },
        'pagination': {'page_size': 1000,
                       'page_after_value': page_after_value}
    }).json()

    if 'next_page_after_value' not in data['pagination'].keys():
        for entry in data['data']:
            try:
                extract_values(entry)
            except KeyError:
                continue
        break
    page_after_value = data['pagination']['next_page_after_value']

    for entry in data['data']:
        try:
            extract_values(entry)
        except KeyError:
            continue

You can verify that the data was fetched correctly by checking the length of of of the lists, e.g., bandgap

In [ ]:
len(bandgaps)

## Save Data

### Put Data into a Pandas DataFrame and Save

In [ ]:
import os

import pandas as pd

df = pd.DataFrame({
    'reduced_formulas': reduced_formulas,
    'iupac_formulas': iupac_formulas,
    'descriptive_formulas': descriptive_formulas,
    'bandgap': bandgaps,
    'voc': vocs,
    'jsc': jscs,
    'ff': ffs,
    'pce': pces,
    'device_stack': device_stack,
    'htl': htl,
    'etl': etl,
    'structural_dimensionality': structural_dimensionality
    })

if not os.path.exists('data'):
    os.makedirs('data')
df.to_csv('data/perovskite_bandgap_devices.csv', index=False)

We can also have a quick look on the DataFrame, and change the units of the bandgap from J to eV.

In [ ]:
df['bandgap'] = pd.to_numeric(df['bandgap'], errors='coerce')
df['bandgap'] = df['bandgap']*6.24150974e18
df.head()

## Load data 

In the previous part, we retrieved perovskite solar cell data using the NOMAD API, converted it to a Pandas DataFrame, and saved it as **perovskite_bandgap_devices.csv** in the **data** folder. If you already have the data, you can start from this section, where we will import and clean it.

The distribution of the chemical diveristy of the dataset can be viewed in the dynamic periodioc table of the NOMAD entries explorer. The down right corner of every element shows the number of entries (solar cells) that contain in the absorber a given element. It becomes obvius that the dtaa set is imbalance with the majority of the entries being *Pb* based, containing *C*, *N* anf *H* (hybrid perovskites) and mostly halide compounds, with most of the entries having *I* and/or *Br*. 

In [ ]:
import pandas as pd

df = pd.read_csv('data/perovskite_bandgap_devices.csv')
df['bandgap'] = pd.to_numeric(df['bandgap'], errors='coerce')
df['bandgap'] = df['bandgap']*6.24150974e18
df.head()

## EDA & Data Cleaning

Exploratory Data Analysis (EDA) is a useful step in understanding and preparing datasets for modeling by summarizing data, checking for anomalies, finding patterns and relationships.

### Insights from NOMAD GUI

The distribution of chemical diversity of the dataset can be explored using the [periodic table in the NOMAD solar cell app](https://nomad-lab.eu/prod/v1/gui/search/solarcells). The bottom-right corner of each element displays the number of entries (solar cells) that include the element in the absorber. The dataset is notably imbalanced, with the majority of entries being Pb-based, containing C, N, H (hybrid perovskites), and halides like I and Br.

Let's have a look on the statistical summary of the dataset:

Let's have a look on the statistical summary of the dataset:

In [ ]:
df.describe()

We can see that the dataset contains a diverse range of bandgap values (1.16 to 3.05 eV). Some parameters such as open circuit voltage (voc) and fill factor (ff), include extreme or zero values, which might need attention for data cleaning.

### Remove NaNs

The dataset might include missing values, let's check if there are any:

In [ ]:
df.isna().sum()

 Let's now keep only the part of the dataframe, where the value for bandgap is not Na.

In [ ]:
df = df[df['bandgap'].notna()]
df = df[df['reduced_formulas'].notna()]

In [ ]:
df.isna().sum()

### Group Repeated Formulas

Even before checking the dataset, we can already guess that are many repeated formulas and bandgap values in the DataFrame. But let's verify it:

In [ ]:
df['reduced_formulas'].value_counts().head(10)


The dataset contains many repeated formulas and corresponding bandgap values. To make sure each formula is unique, we will use the `groupby_formula` function from CrabNet to group entries by their formulas and use the mean values for their bandgap. We make a new final DataFrame for our model, call it **df_reduced_formula** and adjust colum names etc.

Let's rename columns and use the `gourpby_formula` function from CrabNet:

In [ ]:
from crabnet.utils.data import groupby_formula  # type: ignore

# Rename the column 'bandgap' to 'target', and 'reduced_formula' to 'formula'
df.rename(columns={'bandgap': 'target'}, inplace=True)
df.rename(columns={'reduced_formulas': 'formula'}, inplace=True)

# Group repeated formulas and take the mean of the target
df_grouped_formula = groupby_formula(df, how='mean')
df_grouped_formula.head()


Let's check the shape of the DataFrame we would like to use for our ML:

In [ ]:
df_grouped_formula.shape


### Check Element Prevalence 

We will use `pymatviz` ([link to docs](https://pymatviz.janosh.dev/)) for visualizing the element prevalence.

Run the following snippet if the package is not installed in the environment yet, otherwise skip it.

You can simply try:

In [ ]:
! pip install pymatviz


in case of problems, try it from its developer repo:

! pip install git+https://github.com/janosh/pymatviz

Once installed, we import and use it:

In [ ]:
from pymatviz import ptable_heatmap_plotly  # type: ignore

ptable_heatmap_plotly(df_grouped_formula['formula'], log=True, colorscale='BuPu', font_colors='black', fmt='.3g', colorbar=dict(orientation="v", title= 'Element Prevalence'))

As you can see the current data is heavily based on hybrid halide perovskites, so we expect the model to perform better when predicting these materials. Let's continue and build the model in the next section!

## Build and Fit the Model 

We randomize the dataset and split it into training, validation, and test sets in a ratio of 80%, 10%, and 10%, respectively.

In [ ]:
import numpy as np

train_df, val_df, test_df = np.split(
    df_grouped_formula.sample(frac=1, random_state=42),
    [
        int(0.8 * len(df_grouped_formula)),
        int(0.9 * len(df_grouped_formula))
    ]
)

We then fit the model using the CrabNet implementation.

In [ ]:
from crabnet.crabnet_ import CrabNet  # type: ignore

crabnet_bandgap = CrabNet(mat_prop="bandgap", model_name='perovskite_bg_prediction', elem_prop='mat2vec', learningcurve=True)
crabnet_bandgap.fit(train_df, val_df)

## Model evaluation

After training, we evaluate the model using the validation data.

In [ ]:
from crabnet.utils.figures import act_pred  # type: ignore
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Train data
train_df_zeros = pd.DataFrame({"formula": train_df['formula'], "target": [0.0]*len(train_df['formula'])})
train_df_predicted, train_df_predicted_sigma = crabnet_bandgap.predict(train_df_zeros, return_uncertainty=True)

act_pred(train_df['target'], train_df_predicted)
r2 = r2_score(train_df['target'], train_df_predicted)
print(f'R2 score: {r2}')
mse = mean_squared_error(train_df['target'], train_df_predicted)
print(f'MSE: {mse}')
mae = mean_absolute_error(train_df['target'], train_df_predicted)
print(f'MAE: {mae} eV')

for validation data we have:

In [ ]:
# Validation data
val_df_zeros = pd.DataFrame({"formula": val_df['formula'], "target": [0.0]*len(val_df['formula'])})
val_df_predicted, val_df_predicted_sigma = crabnet_bandgap.predict(val_df_zeros, return_uncertainty=True)

act_pred(val_df['target'], val_df_predicted)
r2 = r2_score(val_df['target'], val_df_predicted)
print(f'R2 score: {r2}')
mse = mean_squared_error(val_df['target'], val_df_predicted)
print(f'MSE: {mse}')
mae = mean_absolute_error(val_df['target'], val_df_predicted)
print(f'MAE: {mae} eV')

and finally for test data:

In [ ]:
# Test data

test_df_zeros = pd.DataFrame(({"formula": test_df['formula'], "target": [0.0]*len(test_df['formula'])}))
test_df_predicted, test_df_predicted_sigma = crabnet_bandgap.predict(test_df_zeros, return_uncertainty=True)

act_pred(test_df['target'], test_df_predicted)

r2 = r2_score(test_df['target'], test_df_predicted)
print(f'R2 score: {r2}')
mse = mean_squared_error(test_df['target'], test_df_predicted)
print(f'MSE: {mse}')
mae = mean_absolute_error(test_df['target'], test_df_predicted)
print(f'MAE: {mae} eV')

## Predict Bandgap from Individual Formulas

Now we are ready to run some predictions using our trainned model. 
We will start loading the model just in case you want to start direcly here in a new session and the model weights are available.

In [ ]:
import numpy as np
import pandas as pd  # only if you jump to this cell directly
from crabnet.crabnet_ import CrabNet  # type: ignore
from crabnet.kingcrab import SubCrab  # type: ignore

# Instantiate SubCrab
sub_crab_model = SubCrab()

# Instantiate CrabNet and set its model to SubCrab
crabnet_model = CrabNet()
crabnet_model.model = sub_crab_model

# Load the pre-trained network
file_path = r'perovskite_bg_prediction.pth'
crabnet_model.load_network(file_path)

Then define a function and run it for predicting the bandgap from individual formulas:


In [ ]:
# Function to predict the bandgap of a given formula
def predict_bandgap(formula):
    input_df = pd.DataFrame({"formula": [formula], "target": [0.0]})
    prediction, prediction_sigma = crabnet_bandgap.predict(input_df, return_uncertainty=True)
    return prediction, prediction_sigma

# Main script to take user input and display predictions
while True:
    formula = input("Enter a formula (e.g., CsPbBr3, CH3NH3PbI3) or type 'exit' to quit: ")
    if formula.lower() == 'exit':
        print("Exiting prediction tool. Goodbye!")
        break
    try:
        prediction, prediction_sigma = predict_bandgap(formula)
        print(f"Predicted bandgap: {np.round(prediction[0], 3)} +/- {np.round(prediction_sigma[0], 3)} eV")
    except Exception as e:
        print(f"Error during prediction: {e}")

Alternatively, this interactive widget allows you to input a chemical formula, predict its bandgap using the trained model, and check if the formula exists in the dataset. If it does, the widget displays the average bandgap value used during training.

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
from ipywidgets import Button, HBox, Output, Text, VBox


# Function to predict the bandgap of a given formula
def predict_bandgap(formula):
    val_df = pd.DataFrame({"formula": [formula], "target": [0.0]})
    prediction, prediction_sigma = crabnet_bandgap.predict(val_df, return_uncertainty=True)
    return prediction, prediction_sigma

# Function to check if the formula exists in the dataset
def check_formula_in_dataset(formula):
    if formula in df_grouped_formula['formula'].values:
        avg_bandgap = df_grouped_formula.loc[
            df_grouped_formula['formula'] == formula, 'target'
        ].values[0]
        return avg_bandgap
    else:
        return None

# Setting up the widget interface
formula_input = Text(
    value='',
    placeholder='Enter formula (e.g., CsPbBr3)',
    description='Formula:',
)
predict_button = Button(description='Predict Bandgap', button_style='success')
output = Output()

def on_click(b):
    with output:
        output.clear_output()
        try:
            formula = formula_input.value.strip()
            if not formula:
                print("Please enter a valid chemical formula.")
                return

            # Prediction
            prediction, sigma = predict_bandgap(formula)
            print(f"Predicted Bandgap: {np.round(prediction[0], 3)} ± {np.round(sigma[0], 3)} eV")

            # Dataset check
            avg_bandgap = check_formula_in_dataset(formula)
            if avg_bandgap is not None:
                print(f"The averaged literature bandgap for {formula} is {avg_bandgap:.3f} eV (from dataset).")
            else:
                print(f"The formula '{formula}' is not contained in the dataset.")
        except Exception as e:
            print(f"Error: {e}")

predict_button.on_click(on_click)

display(VBox([HBox([formula_input, predict_button]), output]))